In [ ]:
import matplotlib.pyplot as plt
import dill
import sympy
from tqdm import tqdm
from scipy.sparse import lil_matrix
from scipy.sparse.linalg import spsolve
import gmsh
import numpy as np
from scipy.sparse import csr_matrix, coo_matrix, csc_matrix
import pypardiso

### Load precomputed integrals

In [ ]:
bilinear_form = dill.load(open("../calculations/lagrange_cubic_weak_form_functional_p3", "rb"))
right_hand_side = dill.load(open("../calculations/lagrange_cubic_weak_form_right_part_p3", "rb"))
interpolation_function = dill.load(open("../calculations/lagrange_cubic_basis", "rb"))
mapping_function = dill.load(open("../calculations/lagrange_cubic_mapping_function", "rb"))

### Initialize the domain

In [ ]:
gmsh.initialize()
gmsh.model.add("tri_mesh")

ms = 0.5
# Define points
point_1 = gmsh.model.geo.addPoint(0, 0, 0, ms)
point_2 = gmsh.model.geo.addPoint(1, 0, 0, ms)
point_3 = gmsh.model.geo.addPoint(0, 1, 0, ms)
point_4 = gmsh.model.geo.addPoint(-1, 0, 0, ms)
point_5 = gmsh.model.geo.addPoint(0, -1, 0, ms)

# Define circles
arc_1 = gmsh.model.geo.addCircleArc(point_2, point_1, point_3)
arc_2 = gmsh.model.geo.addCircleArc(point_3, point_1, point_4)
arc_3 = gmsh.model.geo.addCircleArc(point_4, point_1, point_5)
arc_4 = gmsh.model.geo.addCircleArc(point_5, point_1, point_2)

# Define line loop
circle = gmsh.model.geo.addCurveLoop([arc_1, arc_2, arc_3, arc_4])

# Define plane surface
plane = gmsh.model.geo.addPlaneSurface([circle])

gmsh.model.geo.synchronize()

ms = 15
gmsh.model.mesh.setTransfiniteCurve(arc_1, ms)
gmsh.model.mesh.setTransfiniteCurve(arc_2, ms)
gmsh.model.mesh.setTransfiniteCurve(arc_3, ms)
gmsh.model.mesh.setTransfiniteCurve(arc_4, ms)
gmsh.model.mesh.setTransfiniteSurface(plane)

physical_group_curves_tag = gmsh.model.addPhysicalGroup(1, [arc_1, arc_2, arc_3, arc_4], name="Boundary curves")
physical_group_surface_tag = gmsh.model.addPhysicalGroup(2, [plane], name="Surface")


gmsh.model.mesh.generate(2)
gmsh.model.mesh.optimize("UntangleMeshGeometry")
gmsh.model.mesh.setOrder(3)
gmsh.fltk.run()

# Triangles

In [ ]:
boundary_node_tags, boundary_node_coords = gmsh.model.mesh.getNodesForPhysicalGroup(1, physical_group_curves_tag)
surface_node_tags, points = gmsh.model.mesh.getNodesForPhysicalGroup(2, physical_group_surface_tag)
points = points.reshape(-1, 3)[:, :-1]
boundary_node_tags = boundary_node_tags - 1

In [ ]:
element_types, element_tags, element_node_tags = gmsh.model.mesh.getElements()
triangles = [nodes for elem_type, nodes in zip(element_types, element_node_tags) if elem_type == 21][0]
triangles = triangles.reshape(-1, 10)
triangles[:, 3:] = triangles[:, [5, 6, 8, 7, 3, 4, -1]]
triangles = triangles - 1

In [ ]:
vertex_marker_is_boundary = np.zeros(surface_node_tags.shape[0])
vertex_marker_is_boundary[boundary_node_tags] = 1

In [ ]:
x, y = sympy.symbols("x y")

In [ ]:
F = sympy.Float(1)
right_vals = np.zeros((points.shape[0]))
right_vals[:] = sympy.lambdify((x, y), F)(*points.T)

G = sympy.Float(0)
right_vals_G = np.zeros((points.shape[0]))
right_vals_G[:] = sympy.lambdify((x, y), G)(*points.T)

In [ ]:
batched_triangles = points[triangles]

In [ ]:
bilinear_form = bilinear_form(batched_triangles[:, 0, 0],
                                    batched_triangles[:, 0, 1],
                                    batched_triangles[:, 1, 0],
                                    batched_triangles[:, 1, 1],
                                    batched_triangles[:, 2, 0],
                                    batched_triangles[:, 2, 1],
                                    batched_triangles[:, 3, 0],
                                    batched_triangles[:, 3, 1],
                                    batched_triangles[:, 4, 0],
                                    batched_triangles[:, 4, 1],
                                    batched_triangles[:, 5, 0],
                                    batched_triangles[:, 5, 1],
                                    batched_triangles[:, 6, 0],
                                    batched_triangles[:, 6, 1],
                                    batched_triangles[:, 7, 0],
                                    batched_triangles[:, 7, 1],
                                    batched_triangles[:, 8, 0],
                                    batched_triangles[:, 8, 1],
                                    batched_triangles[:, 9, 0],
                                    batched_triangles[:, 9, 1],
                                    )

right_hand_side = right_hand_side(batched_triangles[:, 0, 0],
                                    batched_triangles[:, 0, 1],
                                    batched_triangles[:, 1, 0],
                                    batched_triangles[:, 1, 1],
                                    batched_triangles[:, 2, 0],
                                    batched_triangles[:, 2, 1],
                                    batched_triangles[:, 3, 0],
                                    batched_triangles[:, 3, 1],
                                    batched_triangles[:, 4, 0],
                                    batched_triangles[:, 4, 1],
                                    batched_triangles[:, 5, 0],
                                    batched_triangles[:, 5, 1],
                                    batched_triangles[:, 6, 0],
                                    batched_triangles[:, 6, 1],
                                    batched_triangles[:, 7, 0],
                                    batched_triangles[:, 7, 1],
                                    batched_triangles[:, 8, 0],
                                    batched_triangles[:, 8, 1],
                                    batched_triangles[:, 9, 0],
                                    batched_triangles[:, 9, 1],
                                    right_vals[triangles[:, 0]],
                                    right_vals[triangles[:, 1]],
                                    right_vals[triangles[:, 2]],
                                    right_vals[triangles[:, 3]],
                                    right_vals[triangles[:, 4]],
                                    right_vals[triangles[:, 5]],
                                    right_vals[triangles[:, 6]],
                                    right_vals[triangles[:, 7]],
                                    right_vals[triangles[:, 8]],
                                    right_vals[triangles[:, 9]],
                                        ).squeeze()

In [ ]:
b = np.zeros((points.shape[0]), dtype=np.float64)

idx, local_point, i = np.meshgrid(np.arange(triangles.shape[0]), np.arange(10), np.arange(10), indexing='ij')

row_indices = triangles[idx, local_point].ravel()
col_indices = triangles[idx, i].ravel()
values = bilinear_form[idx, local_point, i].ravel()

mask = (vertex_marker_is_boundary[triangles[idx, local_point]].ravel() == 0) & (vertex_marker_is_boundary[triangles[idx, i]].ravel() == 0)
m_mask = vertex_marker_is_boundary[triangles[idx, local_point]].ravel() == 1

np.subtract.at(b, col_indices[m_mask], right_vals_G[row_indices[m_mask]] * values[m_mask])

V = np.concatenate([np.ones(int(vertex_marker_is_boundary.sum())), values[mask]])
R = np.concatenate([np.where(vertex_marker_is_boundary == 1)[0], row_indices[mask]])
C = np.concatenate([np.where(vertex_marker_is_boundary == 1)[0], col_indices[mask]])

matrix = csc_matrix((V, (R, C)), shape=(points.shape[0], points.shape[0]), dtype=np.float64)

idx, local_point = np.meshgrid(np.arange(triangles.shape[0]), np.arange(10), indexing='ij')

row_indices = triangles[idx, local_point].ravel()
values = right_hand_side[idx, local_point].ravel()
mask = vertex_marker_is_boundary[triangles[idx, local_point]].ravel() == 0
v_mask = vertex_marker_is_boundary == 1

np.subtract.at(b, row_indices[mask], values[mask])

b[v_mask] = right_vals_G[v_mask]

In [ ]:
# plt.imshow(matrix.toarray() != 0)

In [ ]:
F = pypardiso.spsolve(matrix.tocsr(), b)

In [ ]:
param = np.linspace(0, 1, 16)
gridx, gridy = np.meshgrid(param, param, indexing="ij")

mask = (gridy <= 1 - gridx)

gridx = gridx[mask]
gridy = gridy[mask]

In [ ]:
global_x, global_y, global_z = [], [], []

for element in triangles:
    
    x0, x1, x2, x3, x4, x5, x6, x7, x8, x9 = (
                                points[element[0], 0], points[element[1], 0], 
                                points[element[2], 0], points[element[3], 0], 
                                points[element[4], 0], points[element[5], 0],
                                points[element[6], 0], points[element[7], 0],
                                points[element[8], 0], points[element[9], 0],
    )
    y0, y1, y2, y3, y4, y5, y6, y7, y8, y9 = (
                                points[element[0], 1], points[element[1], 1], 
                                points[element[2], 1], points[element[3], 1], 
                                points[element[4], 1], points[element[5], 1],
                                points[element[6], 1], points[element[7], 1],
                                points[element[8], 1], points[element[9], 1],
    )
    
    physical_x, physical_y = mapping_function(
                                                gridx, gridy, 
                                                x0, y0, 
                                                x1, y1, 
                                                x2, y2, 
                                                x3, y3, 
                                                x4, y4, 
                                                x5, y5,
                                                x6, y6,
                                                x7, y7,
                                                x8, y8,
                                                x9, y9,                                                
                                ).squeeze()
    
    physical_z = interpolation_function(
                                        gridx, gridy,
                                        
                                        x0, y0,                                         
                                        x1, y1,                                        
                                        x2, y2,                                                
                                        x3, y3,                                        
                                        x4, y4,                                        
                                        x5, y5,
                                        x6, y6,
                                        x7, y7,
                                        x8, y8,
                                        x9, y9,
                                        
                                        F[element[0]],                                                                     
                                        F[element[1]],                                         
                                        F[element[2]],                                         
                                        F[element[3]],                                         
                                        F[element[4]],                                         
                                        F[element[5]],   
                                        F[element[6]],   
                                        F[element[7]],   
                                        F[element[8]],   
                                        F[element[9]],   
                                                                              
                                        ).squeeze()
    
    global_x.append(physical_x)
    global_y.append(physical_y)
    global_z.append(physical_z)
    
global_x = np.concatenate(global_x)
global_y = np.concatenate(global_y)
global_z = np.concatenate(global_z)

In [ ]:
# import plotly
# plot_data = [
#     plotly.graph_objects.Mesh3d(
#                                 x=gx, 
#                                 y=gy,
#                                 z=gz,                                
#                                 )
#     for gx, gy, gz in zip(global_x, global_y, global_z)
#     ]
# fig = plotly.graph_objects.Figure(data=plot_data)
# fig.update_layout()

In [ ]:
import plotly
plot_data = [
    plotly.graph_objects.Mesh3d(
                                x=global_x, 
                                y=global_y,
                                z=global_z,
                                intensity=global_z                              
                                )
]    
fig = plotly.graph_objects.Figure(data=plot_data)
fig.update_layout()

In [25]:
abs(global_z - (global_x**2 + global_y**2 - 1)/4).max()

2.6538346539449476e-06